# Webscraper Notebook

##### Order
- Scrape Nasdaq IPO Calendar
- Follow Links from Ininital Scrape and scrape company data
- Search Company Stocks scrape first 180 days stock data



#### Start By Importing what we will need

In [9]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pickle
import time
from bs4 import BeautifulSoup
import random

#### Define Functions that may be helpful later

In [10]:
def pickle_file(obj, path):
    file = open(path, 'wb')
    pickle.dump(obj, file)
    file.close()

def load_pickle(path):
    file = open(path, 'rb')
    obj = pickle.load(file)
    file.close()
    return obj

def start_driver(driver_path = './WebDriver/chromedriver'):
    ser = Service(driver_path)
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    
    driver_path = './WebDriver/chromedriver'
    driver = webdriver.Chrome(service = ser, options=chrome_options)
    return driver

#Loading a webpage
# driver.get('https://www.nasdaq.com/market-activity/ipos')

## Scrape Nasdaq IPO Calendar

- Define Scrape Specific Functions
  - Navigate To Starting Date
  - Generate Dictionary from rows in table
  - Navigate to next page
- Define Main Loop
  - Load Driver
  - Iterate Through Months from starting date to end date
- Run

In [3]:
def get_starting_page(driver, starting_date = '01/1995'):
    #Open Date 
    date_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/button'))
        ).click()

    #Set Date Picker Date
    date_picker = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div/input'))
        )
    driver.execute_script(f"arguments[0].value = '{starting_date}';", date_picker)     
        # .setAttribute('value', '01/1999')

    #Navigate to New Page
    apply_date_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div/button[2]'))
        ).click()

def generate_data(driver):
    table = driver.find_element(By.XPATH,
        './/table',{'class':'market-calendar-table__table'})
    html = table.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    
    url_base = 'https://www.nasdaq.com'
    table_data = []
    # print(soup)
    # if soup.find('th').text == '':
    #     return []

    rows = soup.find_all('tr',{'class':'market-calendar-table__row'})
    for row in rows:
        row_data = {}

        th = row.find('th', {'role':'cell'})
        ticker = th.findChild().text
        if not ticker:
            continue

        row_data[th['data-column']] = ticker
        row_data['link'] = url_base + row.find('a')['href']

        cells = row.find_all('td',{'role':'cell'})        
        for cell in cells:
            row_data[cell['data-column']] = cell.findChild().text
        table_data.append(row_data)
    return table_data

# generate_data(driver)

def get_next_calendar_page(driver):
    next_scroll = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, ".//button[@class='time-belt__next']"))
        ).click()

    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, ".//button[@class='time-belt__item']"))
        ).click()

# get_next_calendar_page(driver)


In [6]:
def main_loop(pickup_file = False, starting_date= '01/1995', delay = 10):
    if pickup_file:
        data = load_pickle(pickup_file)
    else:
        data = []

    driver = start_driver('./WebDriver/chromedriver')
    print('Started Driver')

    time.sleep(3)

    driver.get('https://www.nasdaq.com/market-activity/ipos')
    print('Loaded Initial Page')

    # time.sleep(delay)

    get_starting_page(driver, starting_date=starting_date)
    print('Loaded Starting Page')

    date_year = 0
    count = 0
    while date_year <= 2022:
        count+=1
        date_year = int(driver.find_element(By.XPATH, ".//button[@class='time-belt__item']").get_attribute('data-year'))

        data += generate_data(driver)
        pickle_file(data, f'Data/nasdaq_checkpoint_{date_year}')
        print(f'Checkpoint-{date_year}\t(Navigations: {count};\t Rows of Data: {len(data)})',end='\r', flush=True)
        time.sleep(random.randint(10,30))

        get_next_calendar_page(driver)
    
    print('\nAll Done')
    return data

data = main_loop(starting_date='02/1998', pickup_file='Data/nasdaq_checkpoint_1998')



/tmp/ipykernel_60201/412141742.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, options=chrome_options)


Started Driver
Loaded Initial Page
Loaded Starting Page
Checkpoint-2023	(Total: 302;	 Len: 7630)
All Done


## Follow Links from Ininital Scrape and scrape company data

- Define Scrape Specific Functions
  - Load Data From Initial Scrape
  - Get Page from link in data
  - Generate Dictionary for page data
  - Navigate to next page (Row in data)
- Define Main Loop
- Run

In [ ]:
def generate_company_overview_data(driver, data={}):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table_soup = soup.find('table')
    table_rows = table_soup.find_all('tr')
    for row in table_rows:
        data[row.find('th').text] = row.find('td').text
    return data

def generate_company_financials_data(driver, data={}):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table_soup = soup.find('table')
    table_rows = table_soup.find_all('tr')
    for row in table_rows:
        data[row.find('th').text] = row.find('td').text
    filing_table = soup.find_all('tbody')[1]
    filing_row = filing_table.find('tr')
    filing_cells = filing_row.find_all('td')
    data['form_type'] = filing_cells[1].text
    data['filing_date_received'] = filing_cells[2].text
    data['filing_link'] = filing_cells[3].find('a')['href']
    return data



In [37]:
driver = start_driver('./WebDriver/chromedriver')

In [30]:
# driver = start_driver('./WebDriver/chromedriver')

ipo_calendar_data = load_pickle('Data/nasdaq_ipos')

# ipo_data[:5]

data = []
row_data = {}
ticker = ipo_calendar_data[0]['proposedTickerSymbol']
row_data ['proposedTickerSymbol']= ticker
url = ipo_calendar_data[0]['link']




In [38]:
driver.get(ipo_calendar_data[0]['link'])

In [39]:
def navigate_to_financials(driver, url):
    financial_url = url.replace('overview','financial-filings')
    driver.get(financial_url)
    return financial_url

navigate_to_financials(driver, url)

'https://www.nasdaq.com/market-activity/ipos/financial-filings?dealId=1205111-102356'

In [51]:
def generate_company_financials_data(driver, data={}):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table_soup = soup.find('table')
    table_rows = table_soup.find_all('tr')
    for row in table_rows:
        data[row.find('th').text] = row.find('td').text
    filing_table = soup.find_all('tbody')[1]
    filing_row = filing_table.find('tr')
    filing_cells = filing_row.find_all('td')
    data['form_type'] = filing_cells[1].text
    data['filing_date_received'] = filing_cells[2].text
    data['filing_link'] = filing_cells[3].find('a')['href']
    return data

generate_company_financials_data(driver)

{'Revenue': '$145,000.00',
 'Net Income': '-$8,518,000.00',
 'Total Assets': '$6,031,000.00',
 'Total Liabilities': '$2,156,000.00',
 "Stockholders' Equity": '$3,875,000.00',
 'form_type': 'F-1',
 'filing_date_received': '03/07/2022',
 'filing_link': 'http://secfilings.nasdaq.com/filingFrameset.asp?FilingID=15638250&View=html'}

## Search Company Stocks scrape first 180 days stock data

- Define Scrape Specific Functions
  - Load Data From Initial Scrape
  - Generate Search Url for ticker
  - Check if search has valid results
  - Generate Dictionary for page data
  - Navigate to next
- Define Main Loop
- Run